# Testing Models

## (3) Ridge Regression

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset - new data set after feature extraction
train = pd.read_csv("new_feauturesExtracted_dataset.csv")

train.head(10)


,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,neg,neu,pos,compound,Helpfulness,UnHelpfulness,UserAvgScore,NumUppercase_T,NumUppercase_S,ReviewLength,SummaryLength,Month,Year,numExclamation_S,numExclamation_T
0,1561604,1,4,2.0,1388793600,0.153,0.692,0.155,-0.4040,0.250000,0.250000,2.000000,9,2,97,3,1,2014,1,0
1,516128,5,7,5.0,1033171200,0.000,0.727,0.273,0.9231,0.714286,0.714286,3.083333,6,5,43,5,9,2002,1,0
2,1265775,0,0,4.0,1390521600,0.000,0.767,0.233,0.7901,0.000000,0.000000,4.000000,0,0,25,1,1,2014,0,1
3,484323,3,4,4.0,1136332800,0.030,0.771,0.199,0.9829,0.750000,0.750000,4.500000,14,1,126,3,1,2006,1,3
4,158350,1,2,5.0,1372204800,0.000,0.880,0.120,0.4588,0.500000,0.500000,4.600000,0,1,23,2,6,2013,0,0
5,192763,5,8,5.0,1138579200,0.035,0.833,0.132,0.9938,0.625000,0.625000,4.500000,188,27,440,17,1,2006,0,4
6,1060454,0,0,4.0,1403654400,0.111,0.737,0.152,0.4118,0.000000,0.000000,3.500000,9,2,54,4,6,2014,0,0
7,860129,0,6,3.0,1162080000,0.151,0.754,0.095,-0.6306,0.000000,0.000000,4.000000,13,3,46,3,10,2006,0,0
8,1470316,2,5,1.0,1357776000,0.030,0.813,0.157,0.7884,0.400000,0.400000,1.000000,3,1,41,1,1,2013,0,0
9,725990,0,0,5.0,1396828800,0.022,0.800,0.178,0.9504,0.000000,0.000000,5.000000,22,1,87,5,4,2014,0,0


In [ ]:
# we have to split this data into train and test
# We create a new column called `train` which is `True` if the instance should be included in the training by using the numpy random number generator.

train['train'] = np.random.rand(len(train)) < 0.8

df_train = train[train.train == True]
df_test = train[train.train == False]

print(df_train.shape, df_test.shape)

(111871, 21) (27813, 21)


In [ ]:
train.columns

Index(['Id', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
       'neg', 'neu', 'pos', 'compound', 'Helpfulness', 'UnHelpfulness',
       'UserAvgScore', 'NumUppercase_T', 'NumUppercase_S', 'ReviewLength',
       'SummaryLength', 'Month', 'Year', 'numExclamation_S',
       'numExclamation_T', 'train'],
      dtype='object')

In [ ]:
predictors = [ 'neg', 'pos', 'compound', 'Helpfulness', 'UnHelpfulness',
       'UserAvgScore', 'NumUppercase_T', 'NumUppercase_S', 'ReviewLength',
       'SummaryLength', 'Month', 'numExclamation_S',
       'numExclamation_T']
X1_train = df_train[predictors]
X1_test = df_test[predictors]
y_train = df_train['Score']
y_test = df_test['Score']

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler() 
X1_std_train = scaler.fit_transform(X1_train)
X1_std_test = scaler.transform(X1_test)

print(X1_std_train[:,:].std(axis=0), X1_std_train[:,:].mean(axis=0))
print(X1_std_test[:,:].std(axis=0), X1_std_test[:,:].mean(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] [-1.83429792e-16  5.44954159e-16  9.88285165e-17 -5.10656345e-17
 -5.10656345e-17  6.87417117e-16  2.75017658e-17  7.74876543e-18
 -7.11362072e-18  1.86732544e-17  1.10896266e-16 -3.18207498e-17
  1.16866626e-17]
[1.00132295 1.00245453 1.00199637 1.00161431 1.00161431 0.99597676
 0.92911589 1.01128128 1.026146   0.99305992 0.99934064 0.99352476
 1.04313094] [ 0.00049589 -0.00597886 -0.00394505 -0.00286141 -0.00286141  0.00516884
 -0.00340218  0.00091669 -0.00664537 -0.00553916 -0.00482497 -0.00403324
 -0.00782639]


In [ ]:
# cross-validation step

alphas = [0.0001, 0.00005,  0.00001,  0.000001, 0.00000001, 0, 0.05, 0.1, 0.5]

# Fit decision tree models using each alpha value
ridge_models = []
for a in alphas:
    ridge = Ridge(ccp_alpha=a)
    ridge.fit(X1_train[predictors], y_train)
    ridge_models.append(ridge)

# Calculate the training and testing MSEs for each model
train_mses = []
test_mses = []
for ridge in ridge_models:
    y_train_pred = ridge.predict(X1_train[predictors])
    y_test_pred = ridge.predict(X1_test[predictors])
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_mses.append(train_mse)
    test_mses.append(test_mse)
print(argmax(test_mses))
print(argmax(train_mses))

# takes ~40 mins to run
# best alpha is 0.5

In [ ]:
ridge = Ridge(alpha=0.5)   # instantiate
ridge.fit(X1_std_train, y_train) # fit to training data
print("ridge coeff:")
ridge.coef_

ridge coeff:


array([-0.08053884,  0.06081108,  0.1005162 ,  0.03566849,  0.03566849,
        0.85352123,  0.00197838,  0.00687652,  0.0109217 , -0.00260771,
       -0.00539841,  0.01312844,  0.00957688])

In [ ]:
# train MSE
y_train_fit_ridge = ridge.predict(X1_std_train)
mse_train_ridge = np.mean( (y_train - y_train_fit_ridge)**2)
print("Train MSE: ")
print(np.sqrt(mse_train_ridge), mse_train_ridge)

Train MSE: 
0.734910853535766 0.5400939626446681


In [ ]:
# test MSE 
y_test_fit_ridge = ridge.predict(X1_std_test)
mse_test_ridge = np.mean( (y_test - y_test_fit_ridge)**2)
print("Test MSE: ")
print(np.sqrt(mse_test_ridge), mse_test_ridge)

Test MSE: 
0.7268415485574599 0.5282986367094064
